In [1]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

df = pl.read_csv('/home/gnoblit/takehome/codametrix/data/clean/triplet_data.csv')
#pos_df = pl.read_ndjson('/home/gnoblit/takehome/codametrix/data/clean/positive_train_data.ndjson')
#neg_df = pl.read_ndjson('/home/gnoblit/takehome/codametrix/data/clean/negative_train_data.ndjson')

: 

In [9]:
# df = df.with_columns(
#         pl.all().replace({'':None})
#     ) 

print(df.head())

['code_anchor', 'description_anchor', 'code_positive', 'description_positive', 'code_negative', 'description_negative']


: 

In [3]:
# l = []
# for a in df.iter_rows(named=True):
#     l_temp = [a['code'], a['code']]
#     l_temp.sort()
#     l.append(l_temp)

In [6]:
def join_dfs(df, cols: list, join_term: str):
    """Function joins df subsets of cols columns on the join_term and returns unique rows"""
    
    df = df.select(cols)
    
    clone = df.select(cols).clone()
    
    df = df.lazy().join(
        clone.lazy(),
        how='left',
        on=join_term
    ).collect(streaming=True)
    del clone
    print(f'join done {df.shape}')
    print(df.columns)
    
    df = df.filter(~(pl.col('description').eq(pl.col('description_right'))))
    df = df.drop(join_term)
    print(f'filter done: {df.shape}')
    
    df = df.with_columns(
        pl.Series('codes', ' ', dtype=pl.String)
    )

    for a in df.iter_rows(named=True):
        l_temp = list(a['code'], a['code_right'])
        l_temp.sort()
        print(l_temp)
        a['codes'] = '-'.join(l_temp)
        print(a['codes'])

    print(f'done with list: {df.shape}')

    df = df.with_columns(
        positive=pl.lit(True))
    print('done adding list')
    
    #df = df.unique(subset='codes')
    print(f'done with unique: {df.shape}')
    df = df.sort('code')

    return df

laterality_cols = ['code', 'category', 'description']
positives = join_dfs(df, laterality_cols, 'category')

join done (60581898, 5)
['code', 'category', 'description', 'code_right', 'description_right']
filter done: (60485112, 4)
done with list: (60485112, 5)
done adding list
done with unique: (60485112, 6)


In [7]:
print(positives.shape)
positives.sort('code').head()

(60485112, 6)


code,description,code_right,description_right,codes,positive
str,str,str,str,str,bool
"""A00""","""Cholera""","""A000""","""Cholera due to Vibrio cholerae…",""" """,true
"""A00""","""Cholera""","""A001""","""Cholera due to Vibrio cholerae…",""" """,true
"""A00""","""Cholera""","""A009""","""Cholera, unspecified""",""" """,true
"""A000""","""Cholera due to Vibrio cholerae…","""A00""","""Cholera""",""" """,true
"""A000""","""Cholera due to Vibrio cholerae…","""A001""","""Cholera due to Vibrio cholerae…",""" """,true


In [ ]:
print(pos_df.shape)
pos_df['code'].unique().sort()

In [ ]:
print(neg_df.shape)
neg_df.head()

In [ ]:
pos_df.columns, neg_df.columns

In [ ]:
pos_df.shape, neg_df.shape

In [ ]:
def join_dfs(df, cols, join_term):
    df = df.select(cols).join(
        df.select(cols),
        how='left',
        on=join_term
    ).filter(~(pl.col('description').eq(pl.col('description_right')))).drop(join_term)

    df = df.with_columns(
        codes = pl.concat_list('code', 'code_right')
    )
    df = df.with_columns(
        codes = pl.col('codes').list.sort()
    )

    df = df.unique('codes').sort('code')

    return df

laterality_cols = ['code', 'up_to_laterality', 'description']
laterality_df = join_dfs(df, laterality_cols, 'up_to_laterality')


In [ ]:
laterality_df.head()

In [ ]:
df['code'][0]

In [ ]:
def gen_negatives(df, master_df, n: int):
    """
    Subset negatives not section.
    Randomly draw n
    Label as negatives
    """
    negatives = []

    for iter_, el_ in enumerate(df.iter_rows(named=True)):
        subset_df = master_df.filter(
            pl.col('section') != el_['code'][:1]
        ).sample(n)
    
        subset_df = subset_df.select(['code', 'section', 'description'])
        subset_df = subset_df.rename(
            {
                'code':'code_right',
                'description': 'description_right'
            }
        )
        subset_df = subset_df.with_columns(
            positive=pl.lit(False),
            code=pl.lit(el_['code']),
            description=pl.lit(el_['description'])
        )
        
        negatives.append(subset_df.drop('section'))
    return negatives

In [ ]:
t = gen_negatives(df=laterality_df[:3], master_df=df, n=2)

In [ ]:
t

In [ ]:
df.head()

In [ ]:
df.with_columns(
    pl.col('code').str.slice(0,1).alias('section')
)

In [ ]:
train_df.head()

In [ ]:
train_df = train_df.with_columns(
    test = pl.concat_str(
        pl.col('code') + ' ' + pl.col('description')
    ),
    
)

In [ ]:
train_df = train_df.with_columns(
    test_right = pl.concat_str(
        pl.col('code_right') + ' ' + pl.col('description_right')
    ),
    
)

In [ ]:
train_df.head()

In [ ]:

test= pl.read_ndjson(
            '/home/gnoblit/takehome/codametrix/data/clean/raw_icd10.ndjson'
        ).with_columns(pl.col(pl.String).replace('', None))
test.slice(42448, 5)

In [ ]:
test2 = test.with_columns(
    pl.when(
        pl.col('up_to_laterality').eq(pl.col('up_to_location'))
    )
    .then(None)
    .otherwise(pl.col('up_to_laterality'))
    .alias('test')
)

test2.slice(42448, 5)

In [ ]:
location_cols = ['code', 'up_to_location', 'description']
up_to_location_df = df.select(location_cols).join(
        df.select(location_cols),
        how='left',
        on='up_to_location'
    ).filter(~(pl.col('description').eq(pl.col('description_right'))))
up_to_location_df

In [ ]:
df['up_to_laterality'].unique()

In [ ]:
laterality_cols = ['code', 'up_to_laterality', 'description']
up_to_laterality_df = df.select(laterality_cols).join(
        df.select(laterality_cols),
        how='left',
        on='up_to_laterality'
    ).filter(~(pl.col('description').eq(pl.col('description_right'))))
up_to_laterality_df

In [ ]:
category_cols = ['code', 'category', 'description']
category_df = df.select(category_cols).join(
    df.select(category_cols),
    how='left',
    on='category'
).filter(~(pl.col('description').eq(pl.col('description_right'))))
category_df

In [ ]:
df = df.with_columns(
    pl.col('path').str.split('-').alias('path_split')
)
df = df.with_columns(
    pl.col('path_split')[:-1]
)

In [ ]:
df.h

In [ ]:
df.select(['code', 'category', 'description'])

In [ ]:
df.select(['code', 'category', 'description']).join(
    df.select(['code', 'category', 'description']),
    how='left',
    on='category'
)

In [ ]:
df.filter(pl.col('code')=='Z189')['description'].to_list()